In [ ]:
import pandas as pd

In [ ]:
df_bank = pd.read_csv("../data/bank-full.csv", delimiter=";")

In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_bank, test_size=0.25, random_state=123)

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
# Distribution of the target

In [ ]:
train_df["y"].value_counts(normalize=True)

In [ ]:
# Distributions of categorical and numeric features

In [ ]:
categorical_cols = list(train_df.drop(columns=["y"]).select_dtypes(include=["object"]).columns)
numerical_cols = list(train_df.select_dtypes(include=["int64"]).columns)

In [ ]:
import altair as alt
alt.data_transformers.enable("vegafusion")

alt.Chart(train_df).mark_bar().encode(
    x="count()",
    y=alt.Y(alt.repeat()).type("nominal")
).repeat(
    categorical_cols, columns=3
)

In [ ]:
alt.Chart(train_df).mark_bar().encode(
    x=alt.X(alt.repeat()).type("quantitative").bin(maxbins=40),
    y="count()"
).repeat(
    numerical_cols, columns=3
)

In [ ]:
# Correlations between numeric features

In [ ]:
# measure linear relationship
person_corr_df = train_df[numerical_cols].corr("pearson").unstack().reset_index()
person_corr_df.columns = ["num_variable_0", "num_variable_1", "correlation"]

corr_heatmap = alt.Chart(person_corr_df).mark_rect().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    color="correlation:Q"
).properties(
    width=250,
    height=250
)

text = alt.Chart(person_corr_df, title="Pearson Correlation").mark_text().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    text=alt.Text("correlation:Q", format=".2f")
)

corr_heatmap + text

In [ ]:
# measure monotonic (incl. non-linear) relationship
# FIXME: does it matter? If we apply linear models, we're more afraid of collinearity, i.e. linear relationship
spearman_corr_df = train_df[numerical_cols].corr("spearman").unstack().reset_index()
spearman_corr_df.columns = ["num_variable_0", "num_variable_1", "correlation"]

corr_heatmap = alt.Chart(spearman_corr_df, title="Spearman Correlation").mark_rect().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    color="correlation:Q"
).properties(
    width=250,
    height=250
)

text = alt.Chart(spearman_corr_df).mark_text().encode(
    x=alt.X("num_variable_0").title("Numerical Variable"),
    y=alt.Y("num_variable_1").title("Numerical Variable"),
    text=alt.Text("correlation:Q", format=".2f")
)

corr_heatmap + text

In [ ]:
# checking for correlation between pdays and previous
# FIXME: there is no linear relationship here
pdays_prev = alt.Chart(train_df, title="pdays vs previous").mark_point().encode(
    x="pdays",
    y="previous"
)

pdays_prev_clamped = alt.Chart(train_df, title="pdays vs previous (previous <= 50)").mark_point().encode(
    x="pdays",
    y=alt.Y("previous").scale(domain=(0, 50), clamp=True)
)

pdays_prev | pdays_prev_clamped

### Summary and Recommendations from EDA
- Generally, bar charts were created for categorical variables, and histograms for numerical variables to show illustration. Correlation heatmaps based on two different metrics were generated to investigate the relationships between numerical variables. A scatter plot specifically for `pdays` vs `previous` was created.
- Judging from the proportion of each class in the target, the dataset is unbalanced
- `job`, `education`, `contact` and `poutcome` contain unknown values. We do not have enough information on the dataset to impute these values properly. Note that these values are not null values, but strings called "unknown". **# FIXME: it seems only `contact` and `poutcome` have "unknown"**
- Out of the columns mentioned that contain unknown values, `contact` and `poutcome` need to be dropped since they contain too many unknown examples. We cannot just drop the unknowns from these columns since we would be dropping too many examples, especially considering the size of the data. **# FIXME: `poutcome` is the outcome of the previous marketing campaign. Most of the "unknown" is because `previous = 0`. If the person has never been contacted for marketing before, it makes sense to say "unknown" for this field as "previous marketing" doesn't exist. But for those who have been reached out before, this `poutcome` might be informative! On the other hand, I think `contact` could be kept, because not that many "unknown" in it actually. Also, our task is to find out the importance of features. There should be no farm to keep it, but I would expect the result to tell me that this `contact` is not important.**
- `job` and `education` can be kept. We can just drop the unknowns from these features. **# FIXME: they might contain information too?**
- The distributions of `pdays` and `previous` are heavily skewed. These variables are also correlated with 0.99 Spearman correlation score and 0.44 Pearson correlation score.
- However, upon visual inspection with a scatter plot, `pdays` and `previous` do not seem to be too correlated to be an issue. We can keep them both as features.
- Overall recommendations:
   - Drop `contact` and `poutcome`
   - Drop unknown values from `job` and `education`
   - Ordinal encode `education`
   - One-hot encode other categorical variables
   - Standardize numerical columns

In [ ]:
# train_df = train_df.loc[(train_df["job"] != "unknown") & (train_df["education"] != "unknown")]
# test_df = test_df.loc[(test_df["job"] != "unknown") & (test_df["education"] != "unknown")]

# X_train = train_df.drop(columns=["y"])
# y_train = train_df["y"]
# X_test = test_df.drop(columns=["y"])
# y_test = test_df["y"]

In [ ]:
# from sklearn.compose import make_column_transformer
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

# # recommended transformations
# categorical_feats = ["job", "marital", "default", "housing", "loan", "month"]
# ordinal_feats = ["education"]
# drop_feats = ["contact", "poutcome"]
# numerical_feats = numerical_cols

# education_levels = ["primary", "secondary", "tertiary"]

# col_transformer = make_column_transformer((OneHotEncoder(sparse_output=False, drop="if_binary"), categorical_feats),
#                                           (OrdinalEncoder(categories=[education_levels], dtype=int), ordinal_feats),
#                                           ("drop", drop_feats),
#                                           (StandardScaler(), numerical_feats))

### notes
 - In this study, we wanna understand which factors would affect the most for clients' subscription to the term deposit
 - positive label: `y = "yes"`; negative label: `y = "no"`
 - We don't wanna miss any potential clients. Therefore, we wanna lower as much as possible the Type I error / false positive, i.e. clients being identified as subscribed to our term deposit but actually they didn't.
 - We will select a model that gives us a robust **precision**, so that the model could better explain clients' motivation to the subscription. We would put more trust to the feature importance recommended by the model.
 - In below, I didn't demonstrate different `max_depth` trials for the `DecisionTreeClassifier`, because our focus is on feature importance rather than the prediction, and I found that the recommendation from the Decision Tree is very similar to the Logistic Regression. Honestly, I think we could actually consider dropping the Decision Tree and just use Logistic Regression for the feature importance recommendation. I'm open to that (coz I also think that keeping the Decision Tree there would make the analysis looks more fancy which seems a good thing too?)

In [ ]:
X_train, y_train = train_df.drop(columns=["y"]), train_df["y"]
X_test, y_test = test_df.drop(columns=["y"]), test_df["y"]

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

categorical_feats = ["job", "marital", "default", "housing", "loan", "contact", "day", "month", "poutcome"]
ordinal_feats = ["education"]
numeric_feats = ["age", "balance", "duration", "campaign", "previous", "pdays"]
#drop_feats = ["month", "day"]

education_levels = ["unknown", "primary", "secondary", "tertiary"]

preprocessor = make_column_transformer(
    (OneHotEncoder(sparse_output=False, drop="if_binary"), categorical_feats),
    (OrdinalEncoder(categories=[education_levels], dtype=int), ordinal_feats),
    (StandardScaler(), numeric_feats),
#    ("drop", drop_feats)
)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

model_pipes = {
    "Baseline": DummyClassifier(strategy="most_frequent", random_state=522),
    "DecisionTree": make_pipeline(preprocessor, DecisionTreeClassifier(max_depth=5, random_state=522)),
    "LogisticRegression": make_pipeline(preprocessor, LogisticRegression(max_iter=2000, random_state=522)),
}

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score

mod_precision_score = make_scorer(precision_score, zero_division=0)

classification_metrics = {
    "accuracy": "accuracy",
    "precision": mod_precision_score,
    "recall": "recall", 
}
cross_val_results = {}

for name, pipe in model_pipes.items():
    cross_val_results[name] = pd.DataFrame(
        cross_validate(
            pipe, 
            X_train, 
            y_train=="yes", 
            cv=5,
            return_train_score=True, 
            scoring=classification_metrics)
    ).agg(['mean', 'std']).round(3).T


In [ ]:
pd.concat(
    cross_val_results,
    axis='columns'
).xs(
    'mean',
    axis='columns',
    level=1
).style.format(
    precision=2
).background_gradient(
    axis=None
)

### note:
 - `LogisticRegression` has slightly better test precision than the `DecisionTreeClassifier`, but the `LogisticRegression` has a smaller gap between train scores and test scores, so `LogisticRegression` is more likely to generalize.
 - Limitation: there is still room of improvements on the precision
   - In the future, if we ever need to make prediction of the subscription, we could increase the threshold to yield a better precision. So, I guess it's not so much worry here.
   - If we really keen on improving the precision on the model level, I could see RandomForest is a choice, but not everyone of us know RandomForest yet, haha

In [ ]:
lr_pipe = model_pipes['LogisticRegression']

lr_pipe.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, lr_pipe.predict(X_test)))

### note
 - The precision test score is similar to the validation score as well as the train score. Therefore, we would believe that the feature importance conclusion drawn from this model is generalizable to the future.

In [ ]:
categorical_cols = lr_pipe.named_steps['columntransformer'].named_transformers_['onehotencoder'].get_feature_names_out().tolist()
ordinal_cols = lr_pipe.named_steps['columntransformer'].named_transformers_['ordinalencoder'].get_feature_names_out().tolist()
numeric_cols = lr_pipe.named_steps['columntransformer'].named_transformers_['standardscaler'].get_feature_names_out().tolist()

feature_importance = pd.DataFrame({
    'feature': categorical_cols + ordinal_cols + numeric_cols, 
    'coef': lr_pipe.named_steps['logisticregression'].coef_[0].tolist()
})

### note
 - According to the below heatmap, `poutcome`, `month` and `duration` are the top 3 features that are highly related to whether a client would subscribe to the term deposit or not.
   - If `poutcome == "success"`, clients were already experiencing the good services by the bank, so they're more willing to subscribe new products.
   - If `month == "mar"`, I'm not sure why clients tend to accept the marketing and subscribe the term deposit in March, this pattern also exists in the test set, so it's not likely an over-fitting. I thought it might be related to the financial/tax period/bonus release time in Portugal, but it seems the finance year-end in Portugal is December (it could be March in some countries e.g. China). So I still can't make sense of it yet.
   - If `duration` is longer, that means the clients were more interested in the term deposit product and were more likely to stay on the call, the salesperson had time to do more pitching, so increase the chance of successful subscription

In [ ]:
feature_importance.sort_values('coef', ascending=False).style.format(
    precision=3
).background_gradient(
    cmap="PiYG",
    vmin=-2,
    vmax=2,
    axis=None
)

### some draft/playground

In [ ]:
train_df.query('month == "mar"')['y'].value_counts()

In [ ]:
test_df.query('month == "mar"')['y'].value_counts()

In [ ]:
train_df.query('month == "apr"')['y'].value_counts()

In [ ]:
test_df.query('month == "apr"')['y'].value_counts()

In [ ]:
feature_importance.query("~feature.str.startswith('month') & ~feature.str.startswith('day')").sort_values('coef', ascending=False).style.format(
    precision=3
).background_gradient(
    cmap="PiYG",
    vmin=-2,
    vmax=2,
    axis=None
)